In [7]:
import cv2 
import os 
import matplotlib.pyplot as plt 
import matplotlib.patches as patches 
import numpy as np 
from PIL import Image 
 
# Specify the path to your image directory 
image_dir = 'train_updated_titiles/images' 
mask_dir = 'train_updated_titiles/masks' 
 
image_files = sorted([f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))])
mask_files = sorted([f for f in os.listdir(mask_dir) if os.path.isfile(os.path.join(mask_dir, f))])

# Loop through each image file and display it 
for image_file, mask_file in zip(image_files, mask_files):
    if os.path.splitext(image_file)[1] == '.png' and os.path.splitext(mask_file)[1] == '.png':
        image_path = os.path.join(image_dir, image_file)
        mask_path = os.path.join(mask_dir, mask_file)
        # Read the image 
        image = cv2.imread(image_path) 
        mask = cv2.imread(mask_path) 
        
        binary_mask = np.any(mask > 0, axis=-1) 
    
        image[binary_mask] = [255, 0, 0] 
    
        pil_image = Image.fromarray(image) 
        pil_image.show()
        break